<a href="https://colab.research.google.com/github/VictorLequertier/Portfolio/blob/main/BusinessDataManagement_project_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Predict Sales Volumes by season and region based on a product segmentation

In [ ]:
!pip install -r https://raw.githubusercontent.com/snowflakedb/snowflake-connector-python/v2.7.3/tested_requirements/requirements_36.reqs
!pip install snowflake-connector-python 

import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import snowflake.connector
from sklearn.metrics import mean_absolute_error

     |████████████████████████████████| 104 kB 4.3 MB/s 
     |████████████████████████████████| 3.5 MB 24.4 MB/s 
ERROR: Could not find a version that satisfies the requirement dataclasses==0.8 (from versions: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6)
ERROR: No matching distribution found for dataclasses==0.8
     |████████████████████████████████| 16.5 MB 3.8 MB/s 
     |████████████████████████████████| 192 kB 55.8 MB/s 
     |████████████████████████████████| 105 kB 38.8 MB/s 
     |████████████████████████████████| 3.6 MB 43.9 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 2.0 MB 45.1 MB/s 


In [ ]:
ctx = snowflake.connector.connect( 
    user="clement.schubert@edhec.com",
    password="!Cleclaber63!", 
    account='pn72783.eu-central-1',
    role = 'GROUP_ROLE_1',
    warehouse='COMPUTE_WH', 
    database='GROUP_DB_1', 
    schema='HACKATHON'
)

In [ ]:
cs = ctx.cursor() 
try:
    cs.execute("SELECT current_version()") 
    one_row = cs.fetchone() 
    print(one_row[0])
finally: 
    cs.close()

6.7.0


In [ ]:
def fetch_pandas_old(ctx, sql):
    # CTX is the variable containing all the connection parameters # sql correspond at the sql code to be run
    # running the SQL query on Snowflake
    cur = ctx.cursor() 
    cur.execute(sql)
    # getting all the columns names from the query result
    columns = list(map(lambda x : x.name, cur.description)) 
    rows = 0
    while True:
        # fetching the data into set of 50 thousand rows
        dat = cur.fetchmany(50000) 
        if not dat:
            break
        # concating the resulting dataframes
        if rows == 0:
            df = pd.DataFrame(dat, columns=columns)
        else :
            tmp = pd.DataFrame(dat, columns=columns) 
            df = pd.concat([df, tmp])
        rows += df.shape[0] 
        print("Current number of rows inserted is: {}".format(rows))
    cur.close()
    print("--- The total number of rows inserted in our dataframe is: {} ---".format(rows))
    return df

In [ ]:
claudie_base = """
SELECT
*
FROM GROUP_DB_1.HACKATHON.FINAL_CLAUDIE_VOLUME_SALES_PER_SEASON
"""

In [ ]:
claudie_base = """ SELECT *, 'TRAIN' AS TRAIN_TEST FROM GROUP_DB_1.HACKATHON.FINAL_CLAUDIE_VOLUME_SALES_PER_SEASON UNION ALL SELECT *, 'TEST' AS TRAIN_TEST FROM GROUP_DB_1.HACKATHON.FINAL_CLAUDIE_VOLUME_SALES_PER_SEASON_TEST """

In [ ]:
claudie = fetch_pandas_old(ctx, claudie_base)

Current number of rows inserted is: 50000
Current number of rows inserted is: 122419
--- The total number of rows inserted in our dataframe is: 122419 ---


In [ ]:
claudie_data_test = """
SELECT
*
FROM GROUP_DB_1.HACKATHON.FINAL_CLAUDIE_VOLUME_SALES_PER_SEASON_TEST
"""

In [ ]:
claudie_test = fetch_pandas_old(ctx, claudie_data_test)

Current number of rows inserted is: 16896
--- The total number of rows inserted in our dataframe is: 16896 ---


In [ ]:
maje_base = """
SELECT
*
FROM GROUP_DB_1.HACKATHON.FINAL_MAJE_VOLUME_SALES_PER_SEASON
"""

In [ ]:
maje_train = fetch_pandas_old(ctx, maje_base)

Current number of rows inserted is: 50000
Current number of rows inserted is: 141750
--- The total number of rows inserted in our dataframe is: 141750 ---


In [ ]:
maje_data_test = """
SELECT
*
FROM GROUP_DB_1.HACKATHON.FINAL_MAJE_VOLUME_SALES_PER_SEASON_TEST
"""

In [ ]:
maje_test = fetch_pandas_old(ctx, maje_data_test)

Current number of rows inserted is: 28698
--- The total number of rows inserted in our dataframe is: 28698 ---


In [ ]:
sandro_base = """
SELECT
*
FROM GROUP_DB_1.HACKATHON.FINAL_SANDRO_VOLUME_SALES_PER_SEASON
"""

In [ ]:
sandro_train = fetch_pandas_old(ctx, sandro_base)

Current number of rows inserted is: 50000
Current number of rows inserted is: 150000
Current number of rows inserted is: 269046
--- The total number of rows inserted in our dataframe is: 269046 ---


In [ ]:
sandro_data_test = """
SELECT
*
FROM GROUP_DB_1.HACKATHON.FINAL_SANDRO_VOLUME_SALES_PER_SEASON_TEST
"""

In [ ]:
sandro_test = fetch_pandas_old(ctx, sandro_data_test)

Current number of rows inserted is: 36391
--- The total number of rows inserted in our dataframe is: 36391 ---


In [ ]:
claudie_train.head()

,IDBRAND,CODESEASONSALE,REGION,SHELF,DEPARTMENT,FAMILY,SUBFAMILY,IDPRODUCT,CLUSTER_PROF,CLUSTER_GROUP,...,VALFRTYPICALWEAR,VALFRSALESPOTENTIAL,VALFRPARTNERPRICEOK,VALFRORDEREDPRODUCT,VALFRSAISONALITY,VALFRWEAVING,VALFRSUPPLYTYPE,VALFRUNIVERS,MAIN_MATERIALLABEL,MAIN_MAT_PERCENTAGE
0,3,E2019,EUR,WOMEN,BAS,ROBE,MANCHES LONGUES,3-00008965,1.000000,None,...,None,None,None,None,None,None,FACON,None,Viscose,100.00
1,3,H2018,ASIA,WOMEN,HAUTS,CARDIGAN,MANCHES LONGUES,3-00007256,1.000000,None,...,None,None,None,None,None,None,NEGOCE,None,Viscose,51.00
2,3,E2020,ASIA,WOMEN,BAS,ROBE,MANCHES COURTES,3-00007519,1.000000,None,...,None,None,None,None,None,None,FACON,None,Polyester,100.00
3,3,E2020,FRANCE,WOMEN,BAS,ROBE,MANCHES LONGUES,3-00009539,1.000000,2.000000,...,None,None,None,None,None,None,FACON,None,Polyester,100.00
4,3,H2017,EUR,WOMEN,DESSUS,MANTEAU,GENOUX,3-00007419,0.000000,None,...,None,None,None,None,None,None,FACON,None,Polyamide,100.00


In [ ]:
## def select_11_first(df):
    ## return(df.iloc[:,1:10], df.iloc[:,10])

In [ ]:
## (X_claudie_reduced_train, Y_claudie_reduced_train) = select_11_first(claudie_train)
## X_claudie_reduced_test, Y_claudie_reduced_test = select_11_first(claudie_test)

In [ ]:
## X_claudie_reduced_train.head()

,CODESEASONSALE,REGION,SHELF,DEPARTMENT,FAMILY,SUBFAMILY,IDPRODUCT,CLUSTER_PROF,CLUSTER_GROUP
0,E2019,EUR,WOMEN,BAS,ROBE,MANCHES LONGUES,3-00008965,1.000000,None
1,H2018,ASIA,WOMEN,HAUTS,CARDIGAN,MANCHES LONGUES,3-00007256,1.000000,None
2,E2020,ASIA,WOMEN,BAS,ROBE,MANCHES COURTES,3-00007519,1.000000,None
3,E2020,FRANCE,WOMEN,BAS,ROBE,MANCHES LONGUES,3-00009539,1.000000,2.000000
4,H2017,EUR,WOMEN,DESSUS,MANTEAU,GENOUX,3-00007419,0.000000,None


In [ ]:
#Dealing with null values in clusters:
def cleaning_null_clusters(df):
    df = df.copy()
    
    cluster_group_fill_cluster_nbr = len(set(df.loc[:,'CLUSTER_GROUP'])) + 1
    df['CLUSTER_GROUP'].fillna(cluster_group_fill_cluster_nbr, inplace=True)
    cluster_prof_fill_cluster_nbr = len(set(df.loc[:,'CLUSTER_PROF'])) + 1
    df['CLUSTER_PROF'].fillna(cluster_prof_fill_cluster_nbr, inplace=True)
    df['CLUSTER_GROUP'] = df['CLUSTER_GROUP'].astype(int)
    df['CLUSTER_PROF']= df['CLUSTER_PROF'].astype(int)
    return(df)



In [ ]:
claudie_full_clusters_train = cleaning_null_clusters(claudie_train)
claudie_full_clusters_test = cleaning_null_clusters(claudie_test)

In [ ]:
claudie_full_clusters_train['MAIN_ELT_PERC'].iloc[2].split("%")

['100', '']

In [ ]:
##X_claudie_reduced_train_clean = cleaning_null_clusters(X_claudie_reduced_train)
## X_claudie_reduced_test_clean = cleaning_null_clusters(X_claudie_reduced_test)

In [ ]:
def split_cluster_prof_group(df):
    df = df.copy()
    df_prof = df.drop(columns = 'CLUSTER_GROUP')
    df_group = df.drop(columns = 'CLUSTER_PROF')
    return(df_group, df_prof)

In [ ]:
def cleaning_categorical(df):
    df = df.copy()
    #Turning SEASON into IS_SUMMER and YEAR
    df['IS_SUMMER'] = df['CODESEASONSALE'].apply(lambda x : 1 if x[0] == 'E' else 0)
    df['YEAR'] =  df['CODESEASONSALE'].apply(lambda x : int(x[1:]))

    #Dropping useless col
    df.drop(columns = ['IDBRAND','IDPRODUCT','COLORBASE'], inplace = True)

    # setting int values to numbers 
    df["NUMBER_OF_SIZES"] = df["NUMBER_OF_SIZES"].astype(int)
    df['NUMBER_OF_COLORS'] = df['NUMBER_OF_COLORS'].astype(int)
    df['TOTALSALES'] = df['TOTALSALES'].astype(int)
    df['MAX_ITEMWEIGHT'] = df['MAX_ITEMWEIGHT'].astype(float)
    df['MIN_ITEMWEIGHT'] = df['MIN_ITEMWEIGHT'].astype(float)
    df['MAIN_MAT_PERCENTAGE'] = df['MAIN_MAT_PERCENTAGE'].astype(float)

    df['MAIN_ELT_PERC'].fillna("0")

    for i in range(len(df["MAIN_ELT_PERC"])):
      df['MAIN_ELT_PERC'][i] = df['MAIN_ELT_PERC'].iloc[i].split("%")[0]
      
    df['MAIN_ELT_PERC'] =  df['MAIN_ELT_PERC'].astype(float)
    #Dummyfying other categorical columns
    df = pd.get_dummies(df, drop_first = True)
    return(df)

In [ ]:
claudie_dumm_train = cleaning_categorical(claudie_full_clusters_train)
claudie_dumm_test = cleaning_categorical(claudie_full_clusters_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to b

AttributeError: ignored

In [ ]:
claudie_df_dumm.corr()['TOTALSALES']

CLUSTER_PROF                                                                   -0.569377
CLUSTER_GROUP                                                                  -0.195334
TOTALSALES                                                                      1.000000
NUMBER_OF_SIZES                                                                 0.225111
NUMBER_OF_COLORS                                                                0.423915
MAX_ITEMWEIGHT                                                                  0.583265
MIN_ITEMWEIGHT                                                                  0.583265
CARELABEL                                                                       0.195334
MAIN_ELT_PERC                                                                   0.275392
MAIN_MAT_PERCENTAGE                                                             0.566606
IS_SUMMER                                                                      -0.002262
YEAR                 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Y = claudie_df_dumm['TOTALSALES']
X = claudie_df_dumm.drop(columns = 'TOTALSALES')

In [ ]:
from sklearn.preprocessing import StandardScaler

def main_split_df(df_train, df_test):
    # Selecting first columns and separating target
    # (X_brand_reduced_train, Y_brand_reduced_train) = select_11_first(df_train)
    # X_brand_reduced_test, Y_brand_reduced_test = select_11_first(df_test)
    
    #Dealing with null values in clusters
    X_brand_reduced_train_no_null = cleaning_null_clusters(X_brand_reduced_train)
    X_brand_reduced_test_no_null = cleaning_null_clusters(X_brand_reduced_test)

    #Splitting for prof clusters and group clusters
    X_brand_reduced_train_no_null_group, X_brand_reduced_train_no_null_prof = split_cluster_prof_group(X_brand_reduced_train_no_null)
    X_brand_reduced_test_no_null_group, X_brand_reduced_test_no_null_prof = split_cluster_prof_group(X_brand_reduced_test_no_null)

    #Dealing with categorical values
    X_brand_reduced_train_clean_group = cleaning_categorical(X_brand_reduced_train_no_null_group)
    X_brand_reduced_test_clean_group = cleaning_categorical(X_brand_reduced_test_no_null_group)

    X_brand_reduced_train_clean_prof = cleaning_categorical(X_brand_reduced_train_no_null_prof)
    X_brand_reduced_test_clean_prof = cleaning_categorical(X_brand_reduced_test_no_null_prof)

    #Filling non existing columns in testset due to dummyfying
    missing_list_group_cols = list(set(X_brand_reduced_train_clean_group.columns).intersection(set(X_brand_reduced_test_clean_group.columns)))
    X_brand_reduced_train_clean_group = X_brand_reduced_train_clean_group[missing_list_group_cols]
    X_brand_reduced_test_clean_group = X_brand_reduced_test_clean_group[missing_list_group_cols]

    missing_list_prof_cols = list(set(X_brand_reduced_train_clean_prof.columns).intersection(set(X_brand_reduced_test_clean_prof.columns)))
    X_brand_reduced_train_clean_prof = X_brand_reduced_train_clean_prof[missing_list_prof_cols]
    X_brand_reduced_test_clean_prof = X_brand_reduced_test_clean_prof[missing_list_prof_cols]
    
    
    return(X_brand_reduced_train_clean_group, X_brand_reduced_train_clean_prof, X_brand_reduced_test_clean_group, X_brand_reduced_test_clean_prof, Y_brand_reduced_train, Y_brand_reduced_test)

In [ ]:
X_claudie_reduced_train_clean_group, X_claudie_reduced_train_clean_prof, X_claudie_reduced_test_clean_group, X_claudie_reduced_test_clean_prof, Y_claudie_reduced_train, Y_claudie_reduced_test = main_split_df(claudie_train, claudie_test)

In [ ]:
X_claudie_reduced_train_clean_group

array([[ 1.428877  , -0.06220266, -0.05997493, ..., -0.76356039,
        -0.01342155, -0.19083574],
       [-0.6998503 , -0.06220266, -0.05997493, ...,  1.30965411,
        -0.01342155, -0.19083574],
       [ 1.428877  , -0.06220266, -0.05997493, ..., -0.76356039,
        -0.01342155, -0.19083574],
       ...,
       [-0.6998503 , -0.06220266, -0.05997493, ...,  1.30965411,
        -0.01342155, -0.19083574],
       [-0.6998503 , -0.06220266, -0.05997493, ..., -0.76356039,
        -0.01342155, -0.19083574],
       [ 1.428877  , -0.06220266, -0.05997493, ..., -0.76356039,
        -0.01342155, -0.19083574]])

# Model

### Simple OLS linear regression

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import accuracy_score, precision_recall_curve, f1_score, mean_squared_error, confusion_matrix, r2_score

## Sklearn models

### Regressor and penalties regressors

In [ ]:
def test_regression(models, X_train, X_test, y_train, y_test, iterations = 3):
    results = {}
    for model in tqdm(models, colour="GREEN"):
        rmse_trains = []
        rmse_tests = []
        for j in range(iterations):
            model_fitted = models[model].fit(X_train, y_train)
            rmse_trains.append(np.sqrt(mean_squared_error(models[model].fit(X_train, y_train).predict(X_train), y_train)))
            rmse_tests.append(np.sqrt(mean_squared_error(model_fitted.predict(X_test), y_test)))
        print("For {} model the R2 is: ".format(str(models[model])))
        R2 = r2_score(y_test, model_fitted.predict(X_test))
        print(R2)
        print("\n")
        results[model] = [np.mean(rmse_trains), np.mean(rmse_tests)]
    print(pd.DataFrame(results))
    return pd.DataFrame(results), model_fitted

In [ ]:
models = {'OLS': LinearRegression(),
         'Lasso': Lasso(),
         'Ridge': Ridge(),
         'Elasticnet' : ElasticNet()}

In [ ]:
all_datasets = [["Claudie", claudie_train, claudie_test], ["Maje", maje_train, maje_test], ["Sandro", sandro_train, sandro_test]]

performances_models = {}
for df in tqdm(all_datasets):
  # Get brand name
  name_df = df[0]

  print("*******************")
  print(name_df)

  train = df[1]
  test = df[2]

  # Preparing dataset for ML
  X_reduced_train_clean_group, X_reduced_train_clean_prof, X_reduced_test_clean_group, X_reduced_test_clean_prof, Y_reduced_train, Y_reduced_test = main_split_df(train, test)

  print("\n")

  # Testing on prof cluster
  print("Prof clusters performances")
  prof_clust_perf, model_prof = test_regression(models, X_reduced_train_clean_prof, X_reduced_test_clean_prof, Y_reduced_train, Y_reduced_test)
  print("--------------")

  # Testing on our cluster
  print("Our clusters performances")
  our_clust_perf, model_group = test_regression(models, X_reduced_train_clean_group, X_reduced_test_clean_group, Y_reduced_train, Y_reduced_test)

  print("\n")
  print("==================")
  print("\n")

  performances_models[name_df] = {"our_cluster": our_clust_perf, "prof_cluster": prof_clust_perf}





*******************
Claudie


Prof clusters performances
For LinearRegression() model the R2 is: 
-2.6698925076566e+25


For Lasso() model the R2 is: 
0.021311492946121446


For Ridge() model the R2 is: 
-0.12681408648365888


For ElasticNet() model the R2 is: 
0.05244461164394165


            OLS       Lasso       Ridge  Elasticnet
0  1.889724e+02  189.326643  188.946102  190.466309
1  7.291002e+14  139.592649  149.784350  137.354410
--------------
Our clusters performances
For LinearRegression() model the R2 is: 
-1.0822124594156498e+26


For Lasso() model the R2 is: 
0.034931545593797964


For Ridge() model the R2 is: 
-0.07237102530013195


For ElasticNet() model the R2 is: 
0.05503005188023491


            OLS       Lasso       Ridge  Elasticnet
0  1.889111e+02  189.251324  188.807097  190.518266
1  1.467900e+15  138.617916  146.121069  137.166894




*******************
Maje


Prof clusters performances
For LinearRegression() model the R2 is: 
-1.9328435765919681e+25


For Lass

In [ ]:
print(performances_models)

{'Claudie': {'our_cluster':             OLS       Lasso       Ridge  Elasticnet
0  1.889111e+02  189.251324  188.807097  190.518266
1  1.467900e+15  138.617916  146.121069  137.166894, 'prof_cluster':             OLS       Lasso       Ridge  Elasticnet
0  1.889724e+02  189.326643  188.946102  190.466309
1  7.291002e+14  139.592649  149.784350  137.354410}, 'Maje': {'our_cluster':             OLS       Lasso       Ridge  Elasticnet
0  3.655655e+02  365.758595  365.482554  367.569389
1  8.347751e+14  437.884775  438.373420  439.782963, 'prof_cluster':             OLS       Lasso       Ridge  Elasticnet
0  3.651681e+02  365.133800  364.941613  366.909615
1  1.982613e+15  438.387451  439.147954  439.704889}, 'Sandro': {'our_cluster':             OLS       Lasso       Ridge  Elasticnet
0  6.861657e+02  686.469864  686.128962  687.824940
1  9.474568e+14  585.002134  591.794587  585.259047, 'prof_cluster':             OLS       Lasso       Ridge  Elasticnet
0  6.874165e+02  687.503367  687.34

### Cleaning the other variables

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm
from time import sleep
import random
def clean_empty_columns(x):
    if not x:
        return 0
    else: 
        return 1


def clean_other_variables(df):
    df = df.copy()

    df = df.iloc[:, 11:]

    for col in df.columns:
        if df[col].isnull().sum()/df.shape[0] == 1:
            df = df.drop(columns=col)

    col_to_fill = [col for col in df.columns if df[col].isnull().sum()/df.shape[0]>0.3]
    for col in tqdm(col_to_fill, colour="MAGENTA"):
        df[col] = df[col].apply(clean_empty_columns)
    
    df["VALFRPRICESEGMENTATION"] = df["VALFRPRICESEGMENTATION"].apply(clean_empty_columns)
    df["MAIN_ELT_PERC"] = df["MAIN_ELT_PERC"].str.replace("%", "").astype(float).fillna(df["MAIN_ELT_PERC"].str.replace("%", "").astype(float).mean())
    df["MAIN_ELT"] = df["MAIN_ELT"].fillna(lambda x: random.choices(list(claudie_clean["MAIN_ELT"].value_counts(normalize=True).index), weights=list(claudie_clean["MAIN_ELT"].value_counts(normalize=True))))
    df["MAIN_MAT_PERCENTAGE"] = df["MAIN_MAT_PERCENTAGE"].fillna(0)
    df["MAIN_MATERIALLABEL"] = df["MAIN_MATERIALLABEL"].fillna(lambda x: random.choices(list(claudie_clean["MAIN_MATERIALLABEL"].value_counts(normalize=True).index), weights=list(claudie_clean["MAIN_MATERIALLABEL"].value_counts(normalize=True))))

    df = df.fillna(0)

    df = df.drop(columns=["IDCURRENCY"])
    df["NUMBER_OF_SIZES"] = df["NUMBER_OF_SIZES"].astype(int)
    df["MAIN_MAT_PERCENTAGE"] = df["MAIN_MAT_PERCENTAGE"].astype(float)
    df["MIN_ITEMWEIGHT"] = df["MIN_ITEMWEIGHT"].astype(float)
    df["MAX_ITEMWEIGHT"] = df["MAX_ITEMWEIGHT"].astype(float)

    for col in tqdm(df.columns, colour="green"):
        if (str(df.dtypes[col]) not in ["int64", "float64"]):
            dummies_df = pd.get_dummies(df[col], drop_first=True, prefix=col)
            df = df.drop(columns=col)
            df = pd.concat([df, dummies_df], axis=1)

    return df


In [ ]:
# Call clean function
claudie_train_other_variables_clean = clean_other_variables(claudie_train)
claudie_test_other_variables_clean = clean_other_variables(claudie_test)
# Commun columns
claudie_commun_col = list(set(claudie_train_other_variables_clean.columns).intersection(set(claudie_test_other_variables_clean.columns)))
# Filter differents columns
claudie_train_other_variables_clean = claudie_train_other_variables_clean[claudie_commun_col]
claudie_test_other_variables_clean = claudie_test_other_variables_clean[claudie_commun_col]
############

# Call clean function
maje_train_other_variables_clean = clean_other_variables(maje_train)
maje_test_other_variables_clean = clean_other_variables(maje_test)
# Commun columns
maje_commun_col = list(set(maje_train_other_variables_clean.columns).intersection(set(maje_test_other_variables_clean.columns)))
# Filter differents columns
maje_train_other_variables_clean = maje_train_other_variables_clean[maje_commun_col]
maje_test_other_variables_clean = maje_test_other_variables_clean[maje_commun_col]
#############

# Call clean function
sandro_train_other_variables_clean = clean_other_variables(sandro_train)
sandro_test_other_variables_clean = clean_other_variables(sandro_test)
# Common columns
sandro_commun_col = list(set(sandro_train_other_variables_clean.columns).intersection(set(sandro_test_other_variables_clean.columns)))
# Filter differents columns
sandro_train_other_variables_clean = sandro_train_other_variables_clean[sandro_commun_col]
sandro_test_other_variables_clean = sandro_test_other_variables_clean[sandro_commun_col]




# IL FAUT STANDARDIZER je l'avais enlevé juste avant mais il faut le remettre ici

100%|██████████| 47/47 [00:01<00:00, 43.74it/s]


In [ ]:
print(list(claudie_train_other_variables_clean.shape))
print(list(claudie_test_other_variables_clean.shape))
claudie_test_other_variables_clean.head()

[55523, 216]
[16896, 216]


,MAIN_MATERIALLABEL_Rayonne,ORIGINMATERIAL_POLOGNE,DRYFR_PAS DE CONSIGNE DE SÉCHAGE,"DRYCLEANFR_NETTOYAGE PROFESSIONNEL À SEC, AU PC ET AUX HC, TRAITEMENT NORMAL",CODESEASON_H17,"DRYCLEANFR_NETTOYAGE PROFESSIONNEL À SEC, AUX HC, TRAITEMENT MODÉRÉ",MAIN_ELT_LIN,MAIN_MATERIALLABEL_Mouton (peau lainée),MAIN_MATERIALLABEL_Raton Laveur,MAIN_ELT_TRIACÉTATE,...,MAIN_MATERIALLABEL_Vachette,MAIN_ELT_LAINE,ORIGINMATERIAL_BOSNIE-HERZEGOV,MAIN_MATERIALLABEL_Laiton,COLORBASE_True,DRYCLEANFR_PAS DE NETTOYAGE PROFESSIONNEL À L'EAU,MAIN_ELT_SOIE,MAIN_ELT_VACHETTE,MAIN_ELT_VEAU,CODESEASON_H14
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### Concat all variables together and test models

In [ ]:
all_datasets = [
                ["Claudie", 
                 claudie_train, 
                 claudie_test, 
                 claudie_train_other_variables_clean, 
                 claudie_test_other_variables_clean], 
                ["Maje", 
                 maje_train, 
                 maje_test,
                 maje_train_other_variables_clean,
                 maje_test_other_variables_clean], 
                ["Sandro", 
                 sandro_train, 
                 sandro_test,
                 sandro_train_other_variables_clean,
                 sandro_test_other_variables_clean]
                ]

performances_models_all_variables = {}
for df in tqdm(all_datasets):
    # Get brand name
    name_df = df[0]

    print("*******************")
    print(name_df)

    train = df[1]
    test = df[2]

    # Preparing dataset for ML
    X_reduced_train_clean_group, X_reduced_train_clean_prof, X_reduced_test_clean_group, X_reduced_test_clean_prof, Y_reduced_train, Y_reduced_test = main_split_df(train, test)

    # Adding all the variables that we prepared
    X_reduced_train_clean_group = np.concatenate([X_reduced_train_clean_group, df[3]], axis=1)
    X_reduced_train_clean_prof = np.concatenate([X_reduced_train_clean_prof, df[3]], axis=1)
    X_reduced_test_clean_group = np.concatenate([X_reduced_test_clean_group, df[4]], axis=1)
    X_reduced_test_clean_prof = np.concatenate([X_reduced_test_clean_prof, df[4]], axis=1)

    print("\n")

    # Testing on prof cluster
    print("Prof clusters performances")
    prof_clust_perf, model_prof = test_regression(models, X_reduced_train_clean_prof, X_reduced_test_clean_prof, Y_reduced_train, Y_reduced_test)
    print("--------------")

    # Testing on our cluster
    print("Our clusters performances")
    our_clust_perf, model_group = test_regression(models, X_reduced_train_clean_group, X_reduced_test_clean_group, Y_reduced_train, Y_reduced_test)

    print("\n")
    print("==================")
    print("\n")

    performances_models_all_variables[name_df] = {"our_cluster": our_clust_perf, "prof_cluster": prof_clust_perf}


  0%|          | 0/3 [00:00<?, ?it/s]

*******************
Claudie


Prof clusters performances
For LinearRegression() model the R2 is: 
-0.9343904857939449


For Lasso() model the R2 is: 
-0.3094233138817095


For Ridge() model the R2 is: 
-0.9057620584414574


For ElasticNet() model the R2 is: 
-0.006249907477478223


          OLS       Lasso       Ridge  Elasticnet
0  179.514983  183.555167  179.518609  187.995698
1  196.251226  161.465722  194.793582  141.544576
--------------
Our clusters performances
For LinearRegression() model the R2 is: 
-0.9212846621869348


For Lasso() model the R2 is: 
-0.30707616222181366


For Ridge() model the R2 is: 
-0.9005662894745563




 33%|███▎      | 1/3 [00:40<01:21, 40.54s/it]

For ElasticNet() model the R2 is: 
-0.003943584259258559


          OLS       Lasso       Ridge  Elasticnet
0  179.349677  183.335771  179.352452  187.785071
1  195.585278  161.320943  194.527863  141.382273




*******************
Maje


Prof clusters performances
For LinearRegression() model the R2 is: 
-1.3417280707212862e+21


For Lasso() model the R2 is: 
-0.014124379047093072


For Ridge() model the R2 is: 
-0.23326177746192145


For ElasticNet() model the R2 is: 
0.05587652244080543


            OLS       Lasso       Ridge  Elasticnet
0  3.321026e+02  349.848572  332.172284  361.752351
1  1.651855e+13  454.135200  500.803308  438.181414
--------------
Our clusters performances
For LinearRegression() model the R2 is: 
-6.541574316658746e+18


For Lasso() model the R2 is: 
-0.012479281631480044


For Ridge() model the R2 is: 
-0.18658861025570217




 67%|██████▋   | 2/3 [04:19<02:25, 145.66s/it]

For ElasticNet() model the R2 is: 
0.05419744438558283


            OLS       Lasso       Ridge  Elasticnet
0  3.325255e+02  349.606547  332.600710  361.730312
1  1.153402e+12  453.766704  491.235383  438.570883




*******************
Sandro


Prof clusters performances
For LinearRegression() model the R2 is: 
-4.478319068995752e+19


For Lasso() model the R2 is: 
-0.05574477065634875


For Ridge() model the R2 is: 
-0.337492890952763


For ElasticNet() model the R2 is: 
0.05637963576073546


            OLS       Lasso       Ridge  Elasticnet
0  6.310378e+02  645.848685  631.118441  679.126715
1  3.981997e+12  611.396334  688.159488  578.018866
--------------
Our clusters performances
For LinearRegression() model the R2 is: 
-7.294369784570307e+16


For Lasso() model the R2 is: 
-0.0124352514147823


For Ridge() model the R2 is: 
-0.2326713469451711




100%|██████████| 3/3 [16:21<00:00, 327.32s/it]

For ElasticNet() model the R2 is: 
0.059261474927880586


            OLS       Lasso       Ridge  Elasticnet
0  6.321651e+02  646.813238  632.265593  678.128948
1  1.607079e+11  598.724445  660.643265  577.135549






### Hyperparametrisation

In [ ]:
lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03]}
ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

def hyperparametrisation(df, lasso_params, ridge_params):
    models2 = {'OLS': linear_model.LinearRegression(),
            'Lasso': GridSearchCV(linear_model.Lasso(), 
                                param_grid=lasso_params).fit(df[X], df[Y]).best_estimator_,
            'Ridge': GridSearchCV(linear_model.Ridge(), 
                                param_grid=ridge_params).fit(df[X], df[Y]).best_estimator_,}
    return(models2)

### Roc Curve

In [ ]:
y_train_classif = X_train.drop(['TOTALSALES'])
fpr_train, tpr_train, _ = roc_curve(y_train_classif,y_train_prediction)
fpr_test, tpr_test, _ = roc_curve(y_test_classif,y_test_prediction)

plt.plot(fpr_train,tpr_train)
plt.plot(fpr_test,tpr_test)
plt.ylabel("True positive rate")
plt.xlabel("False positive rate")

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
def split_X_Y(df):
    df_X = df.drop(columns="TOTALSALES")
    df_Y = df["TOTALSALES"]
    return df_X, df_Y

In [ ]:
#Dealing with null values in clusters:
def cleaning_null_clusters(df):
    df = df.copy()
    
    cluster_group_fill_cluster_nbr = len(set(df.loc[:,'CLUSTER_GROUP'])) + 1
    df['CLUSTER_GROUP'].fillna(cluster_group_fill_cluster_nbr, inplace=True)
    cluster_prof_fill_cluster_nbr = len(set(df.loc[:,'CLUSTER_PROF'])) + 1
    df['CLUSTER_PROF'].fillna(cluster_prof_fill_cluster_nbr, inplace=True)
    df['CLUSTER_GROUP'] = df['CLUSTER_GROUP'].astype(int)
    df['CLUSTER_PROF']= df['CLUSTER_PROF'].astype(int)
    return(df)

In [ ]:
def split_cluster_prof_group(df):
    df = df.copy()
    df_prof = df.drop(columns = 'CLUSTER_GROUP')
    df_group = df.drop(columns = 'CLUSTER_PROF')
    return(df_group, df_prof)

In [ ]:
def clean_empty_columns(x):
    if not x:
        return 0
    else: 
        return 1

def cleaning_categorical(df):
    df = df.copy()
    # Turning SEASON into IS_SUMMER and YEAR
    df['IS_SUMMER'] = df['CODESEASONSALE'].apply(lambda x : 1 if x[0] == 'E' else 0)
    df['YEAR'] =  df['CODESEASONSALE'].apply(lambda x : int(x[1:]))

    # Dropping useless col
    df.drop(columns = ['IDBRAND','IDPRODUCT','COLORBASE', 'IDCURRENCY'], inplace = True)

    # Deleting a column if the entire columns is None values
    for col in df.columns:
        if df[col].isnull().sum()/df.shape[0] == 1:
            df = df.drop(columns=col)

    # if more than 30% of the column is empty then we fill empty with 0 and all the other flag get converted to 1
    col_to_fill = [col for col in df.columns if df[col].isnull().sum()/df.shape[0]>0.3]
    for col in tqdm(col_to_fill, colour="MAGENTA"):
        df[col] = df[col].apply(clean_empty_columns)
    
    # Special considering for categorical columns
    df["VALFRPRICESEGMENTATION"] = df["VALFRPRICESEGMENTATION"].apply(clean_empty_columns)
    df["MAIN_ELT_PERC"] = df["MAIN_ELT_PERC"].str.replace("%", "").astype(float).fillna(df["MAIN_ELT_PERC"].str.replace("%", "").astype(float).mean())
    df["MAIN_ELT"] = df["MAIN_ELT"].fillna(lambda x: random.choices(list(claudie_clean["MAIN_ELT"].value_counts(normalize=True).index), weights=list(claudie_clean["MAIN_ELT"].value_counts(normalize=True))))
    df["MAIN_MAT_PERCENTAGE"] = df["MAIN_MAT_PERCENTAGE"].fillna(0)
    df["MAIN_MATERIALLABEL"] = df["MAIN_MATERIALLABEL"].fillna(lambda x: random.choices(list(claudie_clean["MAIN_MATERIALLABEL"].value_counts(normalize=True).index), weights=list(claudie_clean["MAIN_MATERIALLABEL"].value_counts(normalize=True))))

    # setting int/float values to numbers 
    df["NUMBER_OF_SIZES"] = df["NUMBER_OF_SIZES"].astype(int)
    df['NUMBER_OF_COLORS'] = df['NUMBER_OF_COLORS'].astype(int)
    df['MAX_ITEMWEIGHT'] = df['MAX_ITEMWEIGHT'].astype(float)
    df['MIN_ITEMWEIGHT'] = df['MIN_ITEMWEIGHT'].astype(float)
    df['MAIN_MAT_PERCENTAGE'] = df['MAIN_MAT_PERCENTAGE'].astype(float)

    # As a last resort we fill the empty values with 0
    df = df.fillna(0)

    # Dummyfying other categorical columns
    for col in tqdm(df.columns, colour="green"):
        if (str(df.dtypes[col]) not in ["int64", "float64"]) and (col!= "TRAIN_TEST"):
            dummies_df = pd.get_dummies(df[col], drop_first=True, prefix=col)
            df = df.drop(columns=col)
            df = pd.concat([df, dummies_df], axis=1)

    # Standardization of the dataset
    df_for_norm = df.drop(columns="TRAIN_TEST")
    df_norm = (df_for_norm-df_for_norm.min())/(df_for_norm.max()-df_for_norm.min())

    return pd.concat([df_norm,df["TRAIN_TEST"]], axis=1)

In [ ]:
def main_split_df(df):

    # Splitting columns and target
    X, Y = split_X_Y(df)
    
    # Dealing with null values in clusters
    X_no_cluster_null = cleaning_null_clusters(X)

    # Splitting for prof clusters and group clusters
    X_no_null_group, X_no_null_prof = split_cluster_prof_group(X_no_cluster_null)

    # Dealing with categorical values
    X_clean_group = cleaning_categorical(X_no_null_group)
    X_clean_prof = cleaning_categorical(X_no_null_prof)

    # Getting train and test
    X_train_group = X_clean_group[X_clean_group["TRAIN_TEST"]=="TRAIN"]
    X_train_prof = X_clean_prof[X_clean_prof["TRAIN_TEST"]=="TRAIN"]
    X_test_group = X_clean_group[X_clean_group["TRAIN_TEST"]=="TRAIN"]
    X_test_prof = X_clean_prof[X_clean_prof["TRAIN_TEST"]=="TEST"]
    Y_train = Y[X_clean_group["TRAIN_TEST"]=="TRAIN"]
    Y_test = Y[X_clean_group["TRAIN_TEST"]=="TEST"]

    # Drop train_test column
    X_train_group.drop(columns="TRAIN_TEST", inplace=True)
    X_train_prof.drop(columns="TRAIN_TEST", inplace=True)
    X_test_group.drop(columns="TRAIN_TEST", inplace=True)
    X_test_prof.drop(columns="TRAIN_TEST", inplace=True)
    
    return(X_train_group, X_train_prof, X_test_group, X_test_prof, Y_train, Y_test)

In [ ]:
X_train_group_claudie, X_train_prof_claudie, X_test_group_claudie, X_test_prof_claudie, Y_train_claudie, Y_test_claudie = main_split_df(claudie)

100%|██████████| 50/50 [00:04<00:00, 10.08it/s]
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
X_train_group_claudie

,CLUSTER_GROUP,NUMBER_OF_SIZES,NUMBER_OF_COLORS,MAX_ITEMWEIGHT,MIN_ITEMWEIGHT,NATURALDRYFR,MAIN_ELT_PERC,SEC_ELT_PERC,SEC_ELT,FULLWASHINGCONDITIONS,...,MAIN_MATERIALLABEL_Viscose,MAIN_MATERIALLABEL_WHITE DUCK,MAIN_MATERIALLABEL_YAK,MAIN_MATERIALLABEL_Yack,MAIN_MATERIALLABEL_Zamac,MAIN_MATERIALLABEL_Zinc,MAIN_MATERIALLABEL_caoutchouc,MAIN_MATERIALLABEL_poils de lapin,MAIN_MATERIALLABEL_Émail,MAIN_MATERIALLABEL_élastique
0,0.000000,0.333333,0.000000,0.088889,0.088889,1.0,0.388889,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.333333,0.000000,0.088889,0.088889,1.0,1.000000,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.333333,0.000000,0.111111,0.111111,0.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.166667,0.500000,0.000000,0.088889,0.088889,1.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.166667,0.166667,0.000000,0.111111,0.111111,0.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20278,1.000000,0.500000,0.000000,0.088889,0.088889,0.0,0.875000,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20279,1.000000,0.500000,0.000000,0.222222,0.222222,0.0,0.972222,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20280,1.000000,0.333333,0.000000,0.222222,0.222222,0.0,0.722222,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20281,0.000000,0.333333,0.166667,0.111111,0.111111,0.0,0.444444,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train_group_claudie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55523 entries, 0 to 20282
Columns: 468 entries, CLUSTER_GROUP to MAIN_MATERIALLABEL_élastique
dtypes: float64(4), int64(25), object(1), uint8(438)
memory usage: 36.3+ MB


In [ ]:
for col in X_train_group_claudie.columns:
    if str(X_train_group_claudie.dtypes[col]).lower() not in ["float64", "int64", "uint8"]:
        print(col)

TRAIN_TEST


In [ ]:
from tqdm import tqdm
from time import sleep
import random
def clean_empty_columns(x):
    if not x:
        return 0
    else: 
        return 1


def clean_other_variables(df):
    df = df.copy()

    df = df.iloc[:, 11:]

    for col in df.columns:
        if df[col].isnull().sum()/df.shape[0] == 1:
            df = df.drop(columns=col)

    col_to_fill = [col for col in df.columns if df[col].isnull().sum()/df.shape[0]>0.3]
    for col in tqdm(col_to_fill, colour="MAGENTA"):
        df[col] = df[col].apply(clean_empty_columns)
    
    df["VALFRPRICESEGMENTATION"] = df["VALFRPRICESEGMENTATION"].apply(clean_empty_columns)
    df["MAIN_ELT_PERC"] = df["MAIN_ELT_PERC"].str.replace("%", "").astype(float).fillna(df["MAIN_ELT_PERC"].str.replace("%", "").astype(float).mean())
    df["MAIN_ELT"] = df["MAIN_ELT"].fillna(lambda x: random.choices(list(claudie_clean["MAIN_ELT"].value_counts(normalize=True).index), weights=list(claudie_clean["MAIN_ELT"].value_counts(normalize=True))))
    df["MAIN_MAT_PERCENTAGE"] = df["MAIN_MAT_PERCENTAGE"].fillna(0)
    df["MAIN_MATERIALLABEL"] = df["MAIN_MATERIALLABEL"].fillna(lambda x: random.choices(list(claudie_clean["MAIN_MATERIALLABEL"].value_counts(normalize=True).index), weights=list(claudie_clean["MAIN_MATERIALLABEL"].value_counts(normalize=True))))

    df = df.fillna(0)

    df = df.drop(columns=["IDCURRENCY"])
    df["NUMBER_OF_SIZES"] = df["NUMBER_OF_SIZES"].astype(int)
    df["MAIN_MAT_PERCENTAGE"] = df["MAIN_MAT_PERCENTAGE"].astype(float)
    df["MIN_ITEMWEIGHT"] = df["MIN_ITEMWEIGHT"].astype(float)
    df["MAX_ITEMWEIGHT"] = df["MAX_ITEMWEIGHT"].astype(float)

    for col in tqdm(df.columns, colour="green"):
        if (str(df.dtypes[col]) not in ["int64", "float64"]):
            dummies_df = pd.get_dummies(df[col], drop_first=True, prefix=col)
            df = df.drop(columns=col)
            df = pd.concat([df, dummies_df], axis=1)

    return df

In [ ]:
# Call clean function
claudie_train_other_variables_clean = clean_other_variables(claudie_train)
claudie_test_other_variables_clean = clean_other_variables(claudie_test)
# Commun columns
claudie_commun_col = list(set(claudie_train_other_variables_clean.columns).intersection(set(claudie_test_other_variables_clean.columns)))
# Filter differents columns
claudie_train_other_variables_clean = claudie_train_other_variables_clean[claudie_commun_col]
claudie_test_other_variables_clean = claudie_test_other_variables_clean[claudie_commun_col]
############

# Call clean function
maje_train_other_variables_clean = clean_other_variables(maje_train)
maje_test_other_variables_clean = clean_other_variables(maje_test)
# Commun columns
maje_commun_col = list(set(maje_train_other_variables_clean.columns).intersection(set(maje_test_other_variables_clean.columns)))
# Filter differents columns
maje_train_other_variables_clean = maje_train_other_variables_clean[maje_commun_col]
maje_test_other_variables_clean = maje_test_other_variables_clean[maje_commun_col]
#############

# Call clean function
sandro_train_other_variables_clean = clean_other_variables(sandro_train)
sandro_test_other_variables_clean = clean_other_variables(sandro_test)
# Common columns
sandro_commun_col = list(set(sandro_train_other_variables_clean.columns).intersection(set(sandro_test_other_variables_clean.columns)))
# Filter differents columns
sandro_train_other_variables_clean = sandro_train_other_variables_clean[sandro_commun_col]
sandro_test_other_variables_clean = sandro_test_other_variables_clean[sandro_commun_col]